In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from nuztf.neutrino_scanner import NeutrinoScanner
from astropy import units as u
from nuwinter.avro import load_avro
from nuwinter.plot import ann_fields, generate_single_page
from nuwinter.utils import deduplicate_df
from nuwinter.paths import get_pdf_path
from pathlib import Path
import pandas as pd
import json
from astropy import units as u
from astropy.coordinates import SkyCoord
from matplotlib.colors import Normalize
from matplotlib.ticker import MultipleLocator
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np
from tqdm import tqdm
from nuztf.skymap_scanner import SkymapScanner







/Users/robertstein/anaconda3/envs/nuwinter_env/lib/python3.11/site-packages/ligo/lw/lsctables.py:89: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(False)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  import lal


In [2]:
base_dir = Path.home().joinpath("Data/winter")

# name = "IC240412A"

name = "S240422ed"

prob_threshold = 0.95
n_days = 7

In [3]:
outpath = str(get_pdf_path(name)).replace("combined", "combined_ztf")
outpath

'/Users/robertstein/Data/nuwinter/S240422ed/winter_candidate_pdf/combined_ztf.pdf'

In [4]:
nu = NeutrinoScanner(name) if "IC" in name else SkymapScanner(
    event=name,
    prob_threshold=prob_threshold,
    n_days=n_days
)

100%|█████████████████████████████████████| 435/435 [00:00<00:00, 248301.88it/s]


In [5]:
ztf_initial_cache = nu.get_initial_cache_path()

download_ztf = False

if not (ztf_initial_cache.exists()):
    download_ztf = True

if ztf_initial_cache.exists():
    if not ztf_initial_cache.stat().st_size > 2:
        download_ztf = True

if download_ztf:
    print(f"No file found at {ztf_initial_cache}")
    if isinstance(nu, SkymapScanner):
        nu.get_alerts()
        nu.filter_alerts()
    else:
        nu.scan_area(t_max=nu.t_min + 14.)

ztf_initial_cache

PosixPath('/Users/robertstein/Data/nuztf/cache/candidates/S240422ed/rev4/0.95/initial_stage.json')

In [6]:
nights = [str(nu.t_min + (i - 1) * u.day).split("T")[0].replace("-", "") for i in range(n_days + 1)]

all_res = []

for night in sorted(nights):
    
    avro_dir = base_dir / f"{night}/avro"

    print(f"Looking for avros in {avro_dir}")
    
    avro_files = list(avro_dir.glob("*.avro"))
    
    for path in avro_files:
        all_res.append(load_avro(path))

if len(all_res) == 0:
    raise FileNotFoundError(f"No avro files found for any night, please download these first using the download notebook!")

res_winter = pd.concat(all_res)

Looking for avros in /Users/robertstein/Data/winter/20240421/avro
Looking for avros in /Users/robertstein/Data/winter/20240422/avro
Looking for avros in /Users/robertstein/Data/winter/20240423/avro
Looking for avros in /Users/robertstein/Data/winter/20240424/avro
Looking for avros in /Users/robertstein/Data/winter/20240425/avro
Looking for avros in /Users/robertstein/Data/winter/20240426/avro
Looking for avros in /Users/robertstein/Data/winter/20240427/avro
Looking for avros in /Users/robertstein/Data/winter/20240428/avro


In [7]:
in_contour = []

for _, row in res_winter.iterrows():
    in_contour.append(nu.in_contour(row["ra"], row["dec"]))

print(f"{np.sum(in_contour)} / {len(in_contour)} candidates in contour")

res_winter = res_winter[in_contour]

2115 / 2115 candidates in contour


In [8]:
res_winter

,objectid,cutout_science,cutout_template,cutout_difference,prv_candidates,candid,deprecated,jd,fid,exptime,...,tmjmag3,tmhmag3,tmkmag3,tmobjectid3,distgaia,plxgaia,ruwegaia,distgaiabright,plxgaiabright,ruwegaiabright
0,WNTR24aadul,b'\x1f\x8b\x08\x00\x1fl\'f\x02\xff\xed\xd7i4Va...,b'\x1f\x8b\x08\x00\x1fl\'f\x02\xff\xed\xd7y4\x...,b'\x1f\x8b\x08\x00\x1fl\'f\x02\xff\xec\x97\xf9...,candid deprecated jd fid expt...,2560,False,2.460424e+06,2,360.0,...,15.642,15.200,15.062,08215558-2248507,2.318287,5.523361,1.076491,4.221504,4.177040,1.004472
1,WNTR24aadum,b'\x1f\x8b\x08\x00\x1fl\'f\x02\xff\xed\xd7i0\x...,b'\x1f\x8b\x08\x00\x1fl\'f\x02\xff\xed\xd7yP\x...,b'\x1f\x8b\x08\x00\x1fl\'f\x02\xff\xec\x97\xf9...,candid deprecated jd fid expt...,2561,False,2.460424e+06,2,360.0,...,16.052,15.525,15.211,08215794-2249009,3.183884,0.088269,1.039886,40.522160,4.177040,1.004472
2,WNTR24aadun,b'\x1f\x8b\x08\x00\x1fl\'f\x02\xff\xed\xd7yTO\...,b'\x1f\x8b\x08\x00\x1fl\'f\x02\xff\xed\x97y4\x...,b'\x1f\x8b\x08\x00\x1fl\'f\x02\xff\xec\x97y4\x...,candid deprecated jd fid expt...,2562,False,2.460424e+06,2,360.0,...,15.693,15.373,15.361,08221335-2248353,1.285336,0.420841,1.076693,64.790749,6.818221,81.451523
3,WNTR24aaduo,b'\x1f\x8b\x08\x00\x1fl\'f\x02\xff\xed\xb7yP\x...,b'\x1f\x8b\x08\x00\x1fl\'f\x02\xff\xed\x97iT\x...,b'\x1f\x8b\x08\x00\x1fl\'f\x02\xff\xec\xb7eTUa...,candid deprecated jd fid expt...,2563,False,2.460424e+06,2,360.0,...,15.973,15.395,15.227,08221045-2249111,3.055140,1.735086,1.075372,46.538082,6.818221,81.451523
4,WNTR24aadup,b'\x1f\x8b\x08\x00\x1fl\'f\x02\xff\xed\x97i4Va...,b'\x1f\x8b\x08\x00\x1fl\'f\x02\xff\xed\x97i4\x...,b'\x1f\x8b\x08\x00\x1fl\'f\x02\xff\xec\x97\xf7...,candid deprecated jd fid expt...,2564,False,2.460424e+06,2,360.0,...,NaN,NaN,NaN,nan,8.056922,0.088269,1.039886,47.579861,4.177040,1.004472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326,WNTR24aabxf,b'\x1f\x8b\x08\x00bl\'f\x02\xff\xec\xbcy4V\xef...,b'\x1f\x8b\x08\x00bl\'f\x02\xff\xec\xbcy4U\xf1...,b'\x1f\x8b\x08\x00bl\'f\x02\xff\xec\xbcgL\x16a...,candid deprecated jd fid expt...,1280,False,2.460424e+06,2,480.0,...,NaN,NaN,NaN,nan,12.954763,0.187098,0.949805,39.230015,16.619865,0.974782
327,WNTR24aabxg,b'\x1f\x8b\x08\x00cl\'f\x02\xff\xec\xb9yT\x8eo...,b'\x1f\x8b\x08\x00cl\'f\x02\xff\xec\xbay4\x96o...,b'\x1f\x8b\x08\x00cl\'f\x02\xff\xecxw8\xd6\x8f...,candid deprecated jd fid expt...,1281,False,2.460424e+06,2,480.0,...,NaN,NaN,NaN,nan,8.670740,0.187098,0.949805,35.436954,16.619865,0.974782
328,WNTR24aabxh,b'\x1f\x8b\x08\x00yl\'f\x02\xff\xec\x97\xf9W\x...,b'\x1f\x8b\x08\x00yl\'f\x02\xff\xec\x97\xf97\x...,b'\x1f\x8b\x08\x00yl\'f\x02\xff\xec\x97y4\x95\...,candid deprecated jd fid expt...,1282,False,2.460424e+06,2,480.0,...,13.386,13.071,12.987,08161149-2458225,2.382803,1.161934,0.973853,66.902260,22.462492,0.999848
329,WNTR24aabxi,b'\x1f\x8b\x08\x00\x7fl\'f\x02\xff\xec\x9aePU\...,b'\x1f\x8b\x08\x00\x7fl\'f\x02\xff\xec\x99y0\x...,b'\x1f\x8b\x08\x00\x7fl\'f\x02\xff\xec\x9ai4\x...,candid deprecated jd fid expt...,1283,False,2.460424e+06,2,480.0,...,15.962,15.858,14.328,08153187-2459286,2.777882,1.329699,1.260201,2.859226,9.164423,1.239988


In [9]:
with open(ztf_initial_cache, "r") as f:
    res_ztf = json.load(f)
    
ztf_sources = []

for entry in res_ztf:

    new_dict = {}
    
    for field in ["candid", "objectId"]:
        new_dict[field] = entry[field]

    for key, val in entry["candidate"].items():
        new_dict[key] = val

    ztf_sources.append(new_dict)

ztf_df = pd.DataFrame(ztf_sources)
ztf_df

,candid,objectId,jd,fid,pid,diffmaglim,programid,isdiffpos,tblid,nid,...,zpmed,clrmed,clrrms,neargaia,neargaiabright,maggaia,maggaiabright,exptime,drb,drbversion
0,2669140023715070003,ZTF24aakxgha,2.460424e+06,2,2669140023715,20.152534,2,f,3,2669,...,28.742001,0.502,0.179616,0.148171,77.994057,15.800767,12.335743,300.0,0.273111,d6_m7
1,2669140025215070048,ZTF24aakxgku,2.460424e+06,2,2669140025215,19.955992,2,f,48,2669,...,28.688999,0.462,0.139645,0.314258,68.333260,15.153409,13.428930,300.0,0.350037,d6_m7
2,2669143614115015033,ZTF24aakvrhi,2.460424e+06,2,2669143614115,20.485649,2,t,33,2669,...,28.868000,0.541,0.152793,0.063140,33.398094,17.630253,8.452576,300.0,0.178979,d6_m7
3,2669140024415070050,ZTF24aakxgac,2.460424e+06,2,2669140024415,20.397671,2,f,50,2669,...,28.806000,0.585,0.160592,0.453580,-999.000000,16.274351,-999.000000,300.0,0.064933,d6_m7
4,2669140025115070012,ZTF24aakxgma,2.460424e+06,2,2669140025115,19.988226,2,f,12,2669,...,28.648001,0.433,0.135750,0.100210,52.195656,17.820419,10.715789,300.0,0.998985,d6_m7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10490,2669256581315010043,ZTF24aakwbas,2.460424e+06,2,2669256581315,19.337290,2,f,43,2669,...,28.504000,0.572,0.198159,1.229177,61.176514,15.458198,8.254787,300.0,0.003531,d6_m7
10491,2669221440615010007,ZTF24aakvzsk,2.460424e+06,2,2669221440615,20.290340,2,f,7,2669,...,28.681999,0.496,0.204546,0.207598,-999.000000,16.927856,-999.000000,300.0,0.999980,d6_m7
10492,2669214243515015005,ZTF24aakvxob,2.460424e+06,2,2669214243515,20.390692,2,t,5,2669,...,28.770000,0.533,0.229554,13.787477,-999.000000,20.119055,-999.000000,300.0,0.238855,d6_m7
10493,2669228631315010043,ZTF24aakvziz,2.460424e+06,2,2669228631315,20.225807,2,f,43,2669,...,28.634001,0.531,0.245592,0.366634,76.707176,17.661245,13.499141,300.0,0.329366,d6_m7


In [10]:
ztf_df

,candid,objectId,jd,fid,pid,diffmaglim,programid,isdiffpos,tblid,nid,...,zpmed,clrmed,clrrms,neargaia,neargaiabright,maggaia,maggaiabright,exptime,drb,drbversion
0,2669140023715070003,ZTF24aakxgha,2.460424e+06,2,2669140023715,20.152534,2,f,3,2669,...,28.742001,0.502,0.179616,0.148171,77.994057,15.800767,12.335743,300.0,0.273111,d6_m7
1,2669140025215070048,ZTF24aakxgku,2.460424e+06,2,2669140025215,19.955992,2,f,48,2669,...,28.688999,0.462,0.139645,0.314258,68.333260,15.153409,13.428930,300.0,0.350037,d6_m7
2,2669143614115015033,ZTF24aakvrhi,2.460424e+06,2,2669143614115,20.485649,2,t,33,2669,...,28.868000,0.541,0.152793,0.063140,33.398094,17.630253,8.452576,300.0,0.178979,d6_m7
3,2669140024415070050,ZTF24aakxgac,2.460424e+06,2,2669140024415,20.397671,2,f,50,2669,...,28.806000,0.585,0.160592,0.453580,-999.000000,16.274351,-999.000000,300.0,0.064933,d6_m7
4,2669140025115070012,ZTF24aakxgma,2.460424e+06,2,2669140025115,19.988226,2,f,12,2669,...,28.648001,0.433,0.135750,0.100210,52.195656,17.820419,10.715789,300.0,0.998985,d6_m7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10490,2669256581315010043,ZTF24aakwbas,2.460424e+06,2,2669256581315,19.337290,2,f,43,2669,...,28.504000,0.572,0.198159,1.229177,61.176514,15.458198,8.254787,300.0,0.003531,d6_m7
10491,2669221440615010007,ZTF24aakvzsk,2.460424e+06,2,2669221440615,20.290340,2,f,7,2669,...,28.681999,0.496,0.204546,0.207598,-999.000000,16.927856,-999.000000,300.0,0.999980,d6_m7
10492,2669214243515015005,ZTF24aakvxob,2.460424e+06,2,2669214243515,20.390692,2,t,5,2669,...,28.770000,0.533,0.229554,13.787477,-999.000000,20.119055,-999.000000,300.0,0.238855,d6_m7
10493,2669228631315010043,ZTF24aakvziz,2.460424e+06,2,2669228631315,20.225807,2,f,43,2669,...,28.634001,0.531,0.245592,0.366634,76.707176,17.661245,13.499141,300.0,0.329366,d6_m7


In [11]:
if len(ztf_df) > 0:
    ztf_positions = SkyCoord(ztf_df["ra"].to_numpy()*u.deg, ztf_df["dec"].to_numpy()*u.deg, frame='icrs')
    
    winter_positions = SkyCoord(res_winter["ra"].to_numpy()*u.deg, res_winter["dec"].to_numpy()*u.deg, frame='icrs')
    idx, d2d, _ = winter_positions.match_to_catalog_sky(ztf_positions)
    
    crossmatch_radius = 3.0 * u.arcsec
    mask = d2d < crossmatch_radius
    
    cut = res_winter[mask]
    cut.reset_index(inplace=True)
    cut

In [12]:
if len(ztf_df) > 0:
    ztf_names = []

    for i in idx[mask]:
        ztf_names.append(ztf_df["objectId"].iloc[i])
    
    cut = cut.assign(ztf_name=ztf_names)
    if len(cut) > 0:
        cut = deduplicate_df(cut)
    cut
else:
    cut = pd.DataFrame()

In [13]:
if len(cut) > 0:
    outpath = str(get_pdf_path(name)).replace("combined", "combined_ztf")
    
    print(f"Saving PDF to {outpath}")
    
    with PdfPages(outpath) as pdf:
        for i, row in tqdm(cut.iterrows(), total=len(cut)):
            generate_single_page(row, ann_fields=["ztf_name"] + ann_fields)
            pdf.savefig()
            plt.close()

    outpath = str(get_pdf_path(name)).replace("combined", "combined_ztf_crossmatched")

    with PdfPages(outpath) as pdf:
        for i, row in tqdm(cut.iterrows(), total=len(cut)):
            generate_single_page(row, ann_fields=["ztf_name"] + ann_fields, crossmatch=True)
            pdf.savefig()
            plt.close()
else:
    print("No candidates found")

Saving PDF to /Users/robertstein/Data/nuwinter/S240422ed/winter_candidate_pdf/combined_ztf.pdf


100%|█████████████████████████████████████████████| 2/2 [00:00<00:00, 14.84it/s]


In [14]:
len(res_winter["objectid"])

2115

In [15]:
crossmatch_radius = 2.0 * u.arcsec

winter_positions = SkyCoord(res_winter["ra"].to_numpy()*u.deg, res_winter["dec"].to_numpy()*u.deg, frame='icrs')


match = []

for i, c in enumerate(winter_positions):
    if i < len(winter_positions) - 1:
        idx, d2d, _ = c.match_to_catalog_sky(winter_positions[i+1:])
        if d2d < crossmatch_radius:
            match.append(idx)
        else:
            match.append(None)
    else:
        match.append(None)

In [16]:
res_winter["match"] = match
res_winter_match = res_winter[pd.notnull(match)]
res_winter_match

,objectid,cutout_science,cutout_template,cutout_difference,prv_candidates,candid,deprecated,jd,fid,exptime,...,tmhmag3,tmkmag3,tmobjectid3,distgaia,plxgaia,ruwegaia,distgaiabright,plxgaiabright,ruwegaiabright,match


In [17]:
all_matched = []

for i, row in res_winter_match.iterrows():
    c = SkyCoord(row["ra"]*u.deg, row["dec"]*u.deg, frame='icrs')
    sep = c.separation(winter_positions)
    mask = (sep.arcsec > 0.) & (sep.arcsec < crossmatch_radius.value)
    matches = res_winter.loc[mask]
    row["ndethist"] = mask.sum()
    row["prv_candidates"] = res_winter.loc[mask]
    print(row["objectid"], res_winter.loc[mask]["objectid"])
    all_matched.append(row)

res_winter_match = pd.concat(all_matched, axis=1).transpose()
res_winter_match

ValueError: No objects to concatenate

In [ ]:
res_winter_match["ndethist"]

In [ ]:
sgscore_cut = 0.5
dist_cut_arcsec = 5.

star_cut = (pd.notnull(res_winter_match["distpsnr1"]) & (res_winter_match["distpsnr1"] < dist_cut_arcsec) & pd.notnull(res_winter_match["sgscore1"]) & (res_winter_match["sgscore1"] > sgscore_cut))

masks = [
    res_winter_match["ndethist"] > 0,
    ~star_cut
]

mask = np.ones(len(res_winter_match), dtype=bool)
for new_mask in masks:
    mask *= new_mask
    
print(f"{np.sum(mask)} / {len(mask)} detections pass all cuts")
cut2 = res_winter_match[mask]

if len(cut2) > 0:
    cut2 = deduplicate_df(cut2)
    cut2.sort_values(by="objectid", inplace=True)
    
    outpath = str(get_pdf_path(name)).replace("combined", "combined_2dets")
    print(f"Saving PDF to {outpath}")
    
    with PdfPages(outpath) as pdf:
        for i, row in tqdm(cut2.iterrows(), total=len(cut2)):
            if i < 20:
                generate_single_page(row, ann_fields=ann_fields, crossmatch=True)
                pdf.savefig()
                plt.close()

In [ ]:
list(res_winter_match.columns)